# 验证和导入项目元数据<a class="anchor" id="top"></a>

在本笔记本中，您将从 `01_Validating_and_Importing_User_Item_Interaction_Data.ipynb` 中上次离开的位置开始构建工作项目元数据集。您这可让您使用筛选器，并且随后会支持 `User Personalization` 或 `HRNN-Metadata` 算法。


要运行本笔记本，您需要运行之前的笔记本 `01_Validating_and_Importing_User_Item_Interaction_Data`，您在后者中创建了一个数据集，并将交互数据导入到了 Amazon Personalize 中。在本笔记本的末尾，您保存了一些变量值，现在您需要将这些值加载到此笔记本中。

In [1]:
%store -r

## 准备项目元数据<a class="anchor" id="prepare"></a>
[返回页首](#top)

接下来要做的是加载数据并确认数据处于良好状态，然后将数据保存到 CSV 中，以便用于 Amazon Personalize。

首先，导入数据科学中常用的 Python 库集合。

In [2]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

接下来，打开数据文件并查看前几行。

In [3]:
original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
original_data.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


此操作并不能真正告诉我们有关数据集的更多信息，因此我们将进一步探索原始信息。我们可以看到，流派信息通常分组在一起，这对我们有益，因为 Personalize 支持这种结构。

In [5]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


在其中，您可以看到数据集中总共有 62000 多个条目（对于完整数据集）/9742 个条目（较小版本数据集），分为 3 列。

这是一个非常小的数据集，仅包含适用于每个条目的电影 ID、标题和流派列表。但是，Movielens 数据集中还有其他可用数据。例如，标题包括电影发行的年份。让我们再看看另一列元数据

In [6]:
original_data['year'] =original_data['title'].str.extract('.*\((.*)\).*',expand = False)
original_data.head(5)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


从项目元数据的角度来看，我们只希望包含与训练模型和/或筛选结果相关的信息，因此我们将删除标题，保留流派信息。

In [7]:
itemmetadata_df = original_data.copy()
itemmetadata_df = itemmetadata_df[['movieId', 'genres', 'year']]
itemmetadata_df.head()

,movieId,genres,year
0,1,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Adventure|Children|Fantasy,1995
2,3,Comedy|Romance,1995
3,4,Comedy|Drama|Romance,1995
4,5,Comedy,1995


操作数据后，请务必确认数据格式是否已更改。

In [8]:
itemmetadata_df.dtypes

movieId     int64
genres     object
year       object
dtype: object

Amazon Personalize 有一个默认 `ITEM_ID` 列，将映射到我们的 `movieId`，现在我们可以通过同时指定 `GENRE`，获得更加充实的更多信息。

In [9]:
itemmetadata_df.rename(columns = {'genres':'GENRE', 'movieId':'ITEM_ID', 'year':'YEAR'}, inplace = True) 

就是这样！此时，数据已准备就绪，我们只需将数据保存为 CSV 文件。

In [10]:
itemmetadata_filename = "item-meta.csv"
itemmetadata_df.to_csv((data_dir+"/"+itemmetadata_filename), index=False, float_format='%.0f')

In [11]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### 创建数据集

首先，定义一个架构，告诉 Amazon Personalize 您正在上载的数据集的类型。根据数据集的类型，架构中需要几个保留关键字和强制关键字。更详细的信息可以在[文档](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)中找到。

在这里，您将为需要 `ITEM_ID` 和 `GENRE` 字段的项目元数据创建架构。它们在架构中的定义顺序必须与它们在数据集中的出现顺序相同。

In [12]:
itemmetadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        },{
            "name": "YEAR",
            "type": "int",
        },
        
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-item",
    schema = json.dumps(itemmetadata_schema)
)

itemmetadataschema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:136455442858:schema/personalize-poc-movielens-item",
  "ResponseMetadata": {
    "RequestId": "221f8e73-aa81-4650-96c0-20e3e5bda29d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 22:29:29 GMT",
      "x-amzn-requestid": "221f8e73-aa81-4650-96c0-20e3e5bda29d",
      "content-length": "96",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


创建架构后，可以在数据集组中创建数据集。请注意，这还不会加载数据。加载数据将在几个步骤后发生。

In [13]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = itemmetadataschema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:136455442858:dataset/personalize-poc-movielens/ITEMS",
  "ResponseMetadata": {
    "RequestId": "c86476eb-21e6-4743-9f26-228be0ff5e03",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 22:29:31 GMT",
      "x-amzn-requestid": "c86476eb-21e6-4743-9f26-228be0ff5e03",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 将数据上载到 S3

现在，您的 Amazon S3 存储桶已经创建，请上载包含我们的用户-项目-交互数据的 CSV 文件。 

In [14]:
itemmetadata_file_path = data_dir + "/" + itemmetadata_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(itemmetadata_filename).upload_file(itemmetadata_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+itemmetadata_filename

## 导入项目元数据<a class="anchor" id="import"></a>
[返回页首](#top)

之前，您创建了数据集组和数据集来存放您的信息，因此现在您将执行一个导入作业，将数据从 S3 存储桶加载到 Amazon Personalize 数据集。 

In [15]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-item-import1",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, itemmetadata_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:136455442858:dataset-import-job/personalize-poc-item-import1",
  "ResponseMetadata": {
    "RequestId": "ca29c3d7-ecd9-441d-a70e-72956182e6ab",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 16 Sep 2020 22:29:37 GMT",
      "x-amzn-requestid": "ca29c3d7-ecd9-441d-a70e-72956182e6ab",
      "content-length": "116",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


在使用数据集之前，导入作业必须处于活跃状态。执行下面的单元格并等待单元格显示 ACTIVE 状态。它每秒检查一次导入作业的状态，时间最长 6 小时。

导入数据可能需要一些时间，具体取决于数据集的大小。在此研讨会中，数据导入作业大约需要 15 分钟。

In [16]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 98.8 ms, sys: 2.05 ms, total: 101 ms
Wall time: 15min 1s


现在，完成此导入后，您可以启用对推荐以及支持 `HRNN-Metadata` 的筛选。运行下面的单元格，然后继续存储一些值，以便在下一个笔记本中使用。完成该单元格后，打开笔记本 `03_Creating_and_Evaluating_Solutions.ipynb` 以继续。

In [17]:
%store items_dataset_arn
%store itemmetadataschema_arn

Stored 'items_dataset_arn' (str)
Stored 'itemmetadataschema_arn' (str)
